# Geo References

In [ ]:
from pathlib import Path
from project_heart.lv import LV
from project_heart.utils import set_jupyter_backend
from project_heart.enums import *

set_jupyter_backend("pythreejs")

sample_file_ideal = Path("../../_static/sample_files/ellipsoid.stl")
lv_ideal = LV.from_file(sample_file_ideal) 

## Apex and Base Regions

Apex and Base are two main regions of the left ventricle that serve as the foundation for several computations and region-identification methods.


The Apex is located near the papillary muscles; it is oriented anteroinferior to the left, overlaid by the left lung and pleura, and near to, but separated from, the right ventricular apex by the interventricular septal wall 
[[1](https://www.sciencedirect.com/science/article/pii/S2214854X20300340)]. On the other hand, the base region is located at the opposite direction of the Apex. It spans from the papillary muscles to the atrioventricular groove plane and it is separated from the left atrium superficially by the atrioventricular groove and posteriorly by the coronary sinus, except when it is occluded by the aorta and pulmonary trunk. [[1](https://www.sciencedirect.com/science/article/pii/S2214854X20300340)]. 

When considering an ellipsoid with its longitudinal axis parallel to the Z axis, the Apex is defined as the 'lowest' region, while the Base is considered as the 'topmost' region. In our implementation, we are able to identify based on two offsets (ql and qh) expressed as percentages of the lowest and highest node along the Z axis, respectively. Our algorithm iterates over 'n' steps while virtually rotating and re-estimating values for apex and base regions until the longitudinal line formed from apex-base is closely aligned with the Z axis. In addition, the only constraint we impose is that the starting geometry must be fairly aligned along the Z axis.

In [ ]:
lv_ideal.identify_regions(geo_type=LV_GEO_TYPES.IDEAL,
                          apex_base_args=dict(ab_ql=0.05, ab_qh=0.97),
                          recompute_apex_base=False # for demo purposes
                          )
lv_ideal.transform_region_to_facet_data(LV_MESH_DATA.APEX_BASE_REGIONS)
lv_ideal.plot("surface", 
        scalars=LV_MESH_DATA.APEX_BASE_REGIONS,
        container="cells",
        categorical=True,
        )

### Apex and Base Reference Nodes; Longitudinal Line and Normal Vector

By calculating the centroid from the Apex and Base regions, we can determine the Apex and Base virtual nodes, respectively. In this context, a virtual node is a derived node that does not exist in the mesh but serves as the basis for references and calculations. The optimal placement for the Base on an ellipsoid is in the center of a circle in a plane formed by the boundary between the Endocardium and Epicardium regions. Similarly, the Apex should be situated at the same axis of normal vector from the 'base plane' and the Apex region's centroid. 

In our implementation, the centroid of the Apex and Base areas is used as it gives a reseonable approximation of the optimal placement for Apex and Base nodes and is applicable to non-ideal geometries.

In [ ]:
lv_ideal.plot_longitudinal_line()

Considering the different implementations across the literature and vendors, we provided an option to follow standard practices proposed by proposed by the European Association of Echocardiography in 2015 [[2](https://core.ac.uk/reader/53744593?utm_source=linkout)]. As described by the authors, the Apex should be considered as the most distant from Base (or manually defined) and located at the Endocardium wall. To apply corrections, we can set the Apex and Base nodes from a specified nodeset:

In [ ]:
lv_ideal.set_apex_and_base_from_nodeset(
    lv_ideal.REGIONS.ENDO, ql=0.001, qh=0.99)
lv_ideal.plot_longitudinal_line()

## Epicardium, Endocardium and Base regions

The Endocardium and Epicardium are the innermost and outermost layers, respectively, of the myocardial wall. In this case, although there is a slightly more sophisticated method to account for non-idealized shapes, the endocardium may be thought of as the union of surfaces with mostly inward normals, whilst the epicardium comprises all other areas.

In [ ]:
# lv_ideal.transform_region_to_facet_data(LV_MESH_DATA.EPI_ENDO, method="max")
lv_ideal.plot("mesh", 
        scalars=LV_MESH_DATA.EPI_ENDO,
        # container="cells",
        categorical=True,
        )

Although we can easily recognize the Myocardium, we must also consider the 'Base' surface, which defines the'middle' layer between the Endocardium and Epicardium and serves as a reference for boundary conditions and other calculations. This is the location where this data is stored:

In [ ]:
lv_ideal.transform_region_to_facet_data(LV_MESH_DATA.SURFS, method="min")
lv_ideal.plot("surface", 
        scalars=LV_MESH_DATA.SURFS,
        container="cells",
        categorical=True,
        )

Note that this information is also accessible through nodesets.